# Daisyworld model using Campo

## Overview and introduction

DaisyWorld was introduced by James Lovelock and Andrew Watson (1983), to
illustrate the Gaia Hypothesis that organisms can create a
self-regulating system by interacting with their surroundings.

In Daisyworld, only two life forms (agents) are present: black daisies
and white daisies. White daisies have a high albedo, reflecting light,
thereby cooling the surface temperature (a field). In contrast, black
daisies have a low albedo, absorbing light, thereby increasing the
surface temperature.

Daisies can only reproduce in a certain temperature range; this is where
the interaction between the organisms (agents) and their surroundings
(field) occurs. As you will see when running the model, this leads to an
equilibrium between black and white daisies under a relatively wide
range of luminosity conditions.

## Setting up the environment

First import Campo and additional required Python modules to run this
notebook.

In [ ]:
import datetime
import enum

from osgeo import gdal, ogr, osr

import pcraster as pcr
import pcraster.framework as pcrfw

import campo
import utils

## Generating the input files

For initialisation of the model we will create two input files, one file
holding the locations of the daisies as x,y coordinate pair, and a
second file describing the extent and discretisation of the area. The
extent will be used for the temperature field, and each raster cell in
the area can hold one daisy.

The input files will be generated when executing the following code
cell. Create e.g. a raster with 10 rows and columns:

In [ ]:
utils.generate_input_files(rows=10, cols=10)

To obtain reproducible results we fix the random seed. You can set the
seed to 0 to get different results each model run.

In [ ]:
seed = 1
pcr.setrandomseed(seed)

## Model scenarios

The model can be run for different scenarios, which reflect different
luminosity conditions. The `our` scenario reflects our current
luminosity conditions. The scenarios `low` and `high` have relatively
low and high luminosity, respectively, compared to current conditions.
In the `ramp` scenario, the luminosity increases and then decreases over
800 time steps. You will later specify the scenario as input to the
model.

In [ ]:
class Scenarios(enum.Enum):
    low = 1
    high = 2
    our = 3
    ramp = 4

## Implementation of the Daisyworld model

In [ ]:
class DaisyWorld(pcrfw.DynamicModel):
    def __init__(self, fraction_white=0.2, fraction_black=0.2, albedo_white=0.75, albedo_black=0.25, surface_albedo=0.4, scenario=Scenarios.our):
        pcrfw.DynamicModel.__init__(self)
        # Framework requires a clone
        # set a dummy
        pcr.setclone(5, 4, 10, 0, 0)
        self.fraction_white = fraction_white / 100
        self.fraction_black = fraction_black / 100
        self.albedo_white = albedo_white
        self.albedo_black = albedo_black
        self.surface_albedo = surface_albedo
        self.scenario = scenario

        # Simple input checking...
        assert self.fraction_white >= 0 and self.fraction_white <= 1, 'fraction_white should be between 0 and 100'
        assert self.fraction_black >= 0 and self.fraction_black <= 1, 'fraction_black should be between 0 and 100'

        assert self.albedo_white >= 0 and self.albedo_white <= 1, 'albedo_white should be between 0 and 1'
        assert self.albedo_black >= 0 and self.albedo_black <= 1, 'albedo_black should be between 0 and 1'

        assert self.surface_albedo >= 0, 'surface_albedo should be larger than 0'

    def initial(self):
        self.daisy_world = campo.Campo()

        # Daisies

        self.plants = self.daisy_world.add_phenomenon('plants')

        self.plants.add_property_set('daisies', 'location_daisies.csv')

        # default init all agents
        self.plants.daisies.age_min = 0
        self.plants.daisies.age_max = 25
        self.plants.daisies.age = self.plants.daisies.age_min

        self.plants.daisies.albedo_init = 1
        self.plants.daisies.breed = -1
        self.plants.daisies.albedo = self.plants.daisies.albedo_init

        self.plants.daisies.mask_dead = -1
        self.plants.daisies.mask_alive = 1
        self.plants.daisies.mask = self.plants.daisies.mask_dead

        self.plants.daisies.lower = 0
        self.plants.daisies.upper = 1
        self.plants.daisies.init = campo.uniform(
            self.plants.daisies.lower, self.plants.daisies.upper, seed)

        self.plants.daisies.w_value = 0
        self.plants.daisies.b_value = 1
        self.plants.daisies.albedo_white = self.albedo_white
        self.plants.daisies.albedo_black = self.albedo_black

        # cover of the world surface of breeds
        self.plants.daisies.whites = self.fraction_white
        self.plants.daisies.blacks = 1 - self.fraction_black
        self.plants.daisies.breed = campo.where(
            self.plants.daisies.init < self.plants.daisies.whites, self.plants.daisies.w_value, self.plants.daisies.breed)
        self.plants.daisies.breed = campo.where(
            self.plants.daisies.init > self.plants.daisies.blacks, self.plants.daisies.b_value, self.plants.daisies.breed)

        self.plants.daisies.albedo = campo.where(
            self.plants.daisies.init < self.plants.daisies.whites, self.plants.daisies.albedo_white, self.plants.daisies.albedo)
        self.plants.daisies.albedo = campo.where(
            self.plants.daisies.init > self.plants.daisies.blacks, self.plants.daisies.albedo_black, self.plants.daisies.albedo)

        self.plants.daisies.age_init = campo.random_integers(
            self.plants.daisies.age_min, self.plants.daisies.age_max, seed)
        self.plants.daisies.age = campo.where(
            self.plants.daisies.init < self.plants.daisies.whites, self.plants.daisies.age_init, self.plants.daisies.age)
        self.plants.daisies.age = campo.where(
            self.plants.daisies.init > self.plants.daisies.blacks, self.plants.daisies.age_init, self.plants.daisies.age)

        # reuse b_value for mask, -1: dead, 1: alive
        self.plants.daisies.mask = campo.where(
            self.plants.daisies.init < self.plants.daisies.whites, self.plants.daisies.b_value, self.plants.daisies.mask)
        self.plants.daisies.mask = campo.where(
            self.plants.daisies.init > self.plants.daisies.blacks, self.plants.daisies.b_value, self.plants.daisies.mask)

        self.plants.daisies.init_breed = self.plants.daisies.breed
        self.plants.daisies.init_mask = self.plants.daisies.mask

        self.plants.daisies.buffersize = 15
        self.plants.daisies.neighbours = campo.get_others(
            self.plants.daisies, self.plants.daisies, self.plants.daisies.buffersize)

        # technical detail
        self.plants.set_epsg(28992)

        # Temperature field
        self.area = self.daisy_world.add_phenomenon('area')
        self.area.add_property_set('extent', 'area_extent.csv')

        self.area.set_epsg(28992)
        self.area.extent.temperature = 0
        self.area.extent.surface_albedo = self.surface_albedo
        self.area.extent.zeros = 0
        self.area.extent.ones = 1
        self.area.extent.window_size = 30
        self.area.extent.default_local_heating = 80
        self.area.extent.fraction = 1 / (campo.windowtotal(self.area.extent.ones,
                                          self.area.extent.window_size) - 1)

        if self.scenario == Scenarios.low:
            self.area.extent.solar_luminosity = 0.6
        elif self.scenario == Scenarios.high:
            self.area.extent.solar_luminosity = 1.4
        elif self.scenario == Scenarios.our:
            self.area.extent.solar_luminosity = 1.0
        elif self.scenario == Scenarios.ramp:
            self.area.extent.solar_luminosity = 0.8

        self.suface_temperature()
        self.area.extent.init_temp = self.area.extent.temperature

        # create the output lue data set
        self.daisy_world.create_dataset("daisy_world.lue")

        # create real time settings for lue
        date = datetime.date(2000, 1, 1)
        time = datetime.time(0, 0)
        start = datetime.datetime.combine(date, time)
        unit = campo.TimeUnit.month
        stepsize = 1
        self.daisy_world.set_time(start, unit, stepsize, self.nrTimeSteps())

        # technical detail to inform lue properties may change over time
        self.plants.daisies.mask.is_dynamic = True
        self.plants.daisies.age.is_dynamic = True
        self.plants.daisies.albedo.is_dynamic = True
        self.plants.daisies.breed.is_dynamic = True
        self.area.extent.temperature.is_dynamic = True
        self.area.extent.solar_luminosity.is_dynamic = True

        # write initial data to disk
        self.daisy_world.write()

    def suface_temperature(self):
        surface_albedo = (1 - self.area.extent.surface_albedo) * \
            self.area.extent.solar_luminosity

        d_albedo = campo.feature_values_to_raster(
            self.area.extent, self.plants.daisies, self.plants.daisies.albedo)
        daisy_albedo = (1 - d_albedo) * self.area.extent.solar_luminosity

        condition = campo.feature_values_to_raster(
            self.area.extent, self.plants.daisies, self.plants.daisies.mask) == 1
        absorbed_luminosity = campo.where(
            condition, daisy_albedo, surface_albedo)

        local_heating = campo.where(absorbed_luminosity > 0,
                                    72 * campo.log(absorbed_luminosity) + 80,
                                    self.area.extent.default_local_heating)

        self.area.extent.temperature = (
            self.area.extent.temperature + local_heating) / 2.0

    def diffuse_temperature(self, ratio=0.5):
        condition = campo.feature_values_to_raster(
            self.area.extent, self.plants.daisies, self.plants.daisies.mask) == 1
        spread = campo.spread(
            condition, self.area.extent.zeros, self.area.extent.ones) < 15

        # Sum of temperatures neighbours, exclude yourself
        temp_neighbours = campo.windowtotal(
            self.area.extent.temperature, self.area.extent.window_size) - self.area.extent.temperature
        temp_neighbours = campo.where(
            spread, temp_neighbours, self.area.extent.zeros)

        self.area.extent.temperature = (
            1 - ratio) * self.area.extent.temperature + temp_neighbours * self.area.extent.fraction * ratio

    def solar_activity(self, timestep):
        if self.scenario == Scenarios.ramp:
            if timestep > 200 and timestep <= 400:
                self.area.extent.solar_luminosity += 0.005

            if timestep > 500 and timestep <= 750:
                self.area.extent.solar_luminosity -= 0.0025

    def spread_plants(self):
        seed_threshold = 0.1457 * self.area.extent.temperature - \
            0.0032 * self.area.extent.temperature**2 - 0.6443

        self.plants.daisies.init = campo.uniform(
            self.plants.daisies.lower, self.plants.daisies.upper, seed)

        campo.spread_neighbours(self.plants.daisies.neighbours, seed_threshold, self.plants.daisies.init,
                                self.plants.daisies.breed, self.plants.daisies.mask, self.plants.daisies.albedo, self.plants.daisies.age, seed)

    def dynamic(self):
        # Processes executed per time step
        self.suface_temperature()
        self.diffuse_temperature()
        self.spread_plants()
        self.solar_activity(self.currentTimeStep())

        # Increase age of daisies
        self.plants.daisies.age = campo.where(campo.logical_and(self.plants.daisies.age <= self.plants.daisies.age_max,
                                                                self.plants.daisies.mask == 1), self.plants.daisies.age + 1, self.plants.daisies.age_min)

        # Reset mask and albedo for dead plants
        self.plants.daisies.mask = campo.where(
            (self.plants.daisies.age > self.plants.daisies.age_max), self.plants.daisies.mask_dead, self.plants.daisies.mask)
        self.plants.daisies.albedo = campo.where(
            self.plants.daisies.mask == 1, self.plants.daisies.albedo, self.plants.daisies.albedo_init)

        # Write data for current time step to disk
        self.daisy_world.write(self.currentTimeStep())

### Running the model

You can run the model with default parameter settings, or adapt those to
your own needs. Start e.g. with running the four different scenarios.
800 time steps are recommended for the `ramp` scenario, other scenarios
can use fewer time steps.

When the model is running you will see some `.` indicating the progress
over time steps.

In [ ]:
# This might need a moment to complete
if __name__ == '__main__':
    # Time steps  to run
    timesteps = 150
    # Initial fractions of daisy breeds, 0 - 100
    fraction_white = 20
    fraction_black = 20
    # Fraction of daisy albledos, 0 - 1
    albedo_white = 0.75
    albedo_black = 0.25
    # Albedo of the area
    surface_albedo = 0.4
    # Scenatio to run, see above
    scenario = Scenarios.our

    myModel = DaisyWorld(fraction_white, fraction_black, albedo_white, albedo_black, surface_albedo, scenario)
    dynFrw = pcrfw.DynamicFramework(myModel, timesteps)
    dynFrw.run()

## Model ouput

During model run data is written for each time step to the
`daisy_world.lue` output file. This dataset can be used to create
animations and time series plots.

You can visualize your input luminosity over time with:

In [ ]:
utils.plot_luminosity()

To evaluate the resulting spatio-temporal pattern of daisies and the
surface temperature you can animate those with:

In [ ]:
# This might need a moment to complete
utils.animate()

The resulting average surface temperature over time can be plotted with:

In [ ]:
utils.plot_temperature()

The total sizes of the black and white daisy populations over time can
be plotted with:

In [ ]:
utils.plot_population()